In [1]:
import os
import logging
import joblib
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from datetime import datetime
from preprocessing.text_processor import TextProcessor

In [2]:
data_path = "data/labeled/reviews.csv"
teencode_path = "resources/teencode.csv"
stopwords_path = "resources/stopwords.txt"
phrases_path = "resources/phrase_mapping.csv"
output_path = "models"

In [3]:
# Initialize text processor
processor = TextProcessor(
    teencode_path=teencode_path,
    stopword_path=stopwords_path,
    phrase_mapping_path=phrases_path,
)

In [4]:
processor.preprocess("tất cả rất okk")

'tất_cả rất ổn'

In [5]:
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer

# Tải mô hình và tokenizer
model = RobertaForSequenceClassification.from_pretrained(
    "wonrax/phobert-base-vietnamese-sentiment"
)
tokenizer = AutoTokenizer.from_pretrained(
    "wonrax/phobert-base-vietnamese-sentiment", use_fast=False
)

In [6]:
# import torch
# from transformers import RobertaForSequenceClassification, AutoTokenizer
# from underthesea import word_tokenize  # Hoặc dùng pyvi nếu bạn thích

# comments = [
#     "Giao hàng nhanh, sản phẩm đúng như mô tả.",
#     "Sách bị rách và giao sai hàng.",
#     "Chất lượng ổn, giá cả hợp lý.",
#     "Thái độ nhân viên không tốt.",
#     "Đóng gói cẩn thận, rất hài lòng.",
#     "Xuất sắc",
# ]

# # Mapping ID → Label
# id2label = {0: "neg", 1: "pos", 2: "neu"}

# print("📦 Dự đoán cảm xúc:")
# for cmt in comments:
#     segmented = word_tokenize(cmt, format="text")  # Tách từ bằng underthesea
#     input_ids = torch.tensor([tokenizer.encode(segmented)])
#     with torch.no_grad():
#         outputs = model(input_ids)
#         probs = outputs.logits.softmax(dim=-1).squeeze()
#         top_id = torch.argmax(probs).item()
#     label = id2label[top_id]
#     print(f'🗨️ "{cmt}" → 📌 {label}')

In [7]:
df = pd.read_csv(data_path)

In [8]:
df.head()

,productID,userId,rating,comment,label
0,74021317,7991785,5,Một quyển sách hay,pos
1,187827003,18150739,5,"Mình đã từng đọc sơ nội dung sách, rất hay, rấ...",pos
2,271380890,497788,5,"Quyển sách đẹp về hình thức, nội dung mới đọc ...",pos
3,74021317,19165924,5,"Sách đẹp, hài lòng",pos
4,105483727,10170816,5,"sách đóng gói cẩn thận, giao hành nhanh",pos


In [9]:
# from sklearn.metrics import classification_report
# import torch

# true = []
# pred = []

# label2id = {"neg": 0, "pos": 1, "neu": 2}
# id2label = {v: k for k, v in label2id.items()}

# for row in df.itertuples(index=False, name=None):
#     prdId, userId, rating, comment, label = row

#     # Tokenize with proper padding and truncation
#     inputs = tokenizer(comment, return_tensors="pt", truncation=True, padding=True)

#     with torch.no_grad():
#         outputs = model(**inputs)
#         probs = outputs.logits.softmax(dim=-1).squeeze()
#         top_id = torch.argmax(probs).item()

#     pred_label = id2label[top_id]

#     # Append ground truth and prediction
#     true.append(label2id[label.lower()])
#     pred.append(label2id[pred_label])

# # Print evaluation report
# print(classification_report(true, pred, target_names=["neg", "pos", "neu"]))

In [10]:
df = df[df['label'] != 'neu'].dropna(subset='comment')
df.reset_index(drop=True)

,productID,userId,rating,comment,label
0,74021317,7991785,5,Một quyển sách hay,pos
1,187827003,18150739,5,"Mình đã từng đọc sơ nội dung sách, rất hay, rấ...",pos
2,271380890,497788,5,"Quyển sách đẹp về hình thức, nội dung mới đọc ...",pos
3,74021317,19165924,5,"Sách đẹp, hài lòng",pos
4,105483727,10170816,5,"sách đóng gói cẩn thận, giao hành nhanh",pos
...,...,...,...,...,...
4019,47161879,8006396,1,Gửi 2 quyển giống y hệt nhau trong khi mình đặ...,neg
4020,4080373,11079688,1,Mua tiki rất nhiều lần mà đóng gói vậy là thấy...,neg
4021,3953475,7940086,1,Nhận thiếu 1 quyển. Tiki thu hồi lại từ ngày 1...,neg
4022,3953475,6206297,1,Cách đóng gói không thể chấp nhận được,neg


In [11]:
from sklearn.metrics import classification_report
import torch
import pandas as pd

true = []
pred = []

label2id = {"neg": 0, "pos": 1, "neu": 2}
id2label = {v: k for k, v in label2id.items()}

wrong_predictions = [] 

for row in df.itertuples(index=False, name=None):
    prdId, userId, rating, comment, label = row

    # processed = processor.preprocess(comment)
    # Tokenize
    inputs = tokenizer(comment, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        outputs = model(**inputs)
        probs = outputs.logits.softmax(dim=-1).squeeze()
        top_id = torch.argmax(probs).item()

    pred_label = "pos" if id2label[top_id] == "neu" else id2label[top_id]

    # Append ground truth và prediction
    true_label_id = label2id[label.lower()]
    pred_label_id = label2id[pred_label]

    true.append(true_label_id)
    pred.append(pred_label_id)

    if pred_label_id != true_label_id:
        wrong_predictions.append(
            {
                "prdId": prdId,
                "userId": userId,
                "rating": rating,
                "comment": comment,
                "true_label": label.lower(),
                "predicted_label": pred_label,
            }
        )

print(classification_report(true, pred, target_names=["neg", "pos"]))

df_wrong = pd.DataFrame(wrong_predictions)
df_wrong.to_csv(
    "misclassified_reviews.csv", index=False, encoding="utf-8-sig"
)

              precision    recall  f1-score   support

         neg       0.92      0.79      0.85      2113
         pos       0.80      0.92      0.85      1911

    accuracy                           0.85      4024
   macro avg       0.86      0.85      0.85      4024
weighted avg       0.86      0.85      0.85      4024

